In [4]:
import re

def parse_facts(file_path):
    facts = {}
    target = None
    with open(file_path, 'r') as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            if line.startswith("target:"):
                target_line = line[len("target:"):].strip()
                key, op, val = parse_condition(target_line)
                target = (key, val)  # Only store key and expected value
                continue
            if '=' in line:
                key, val = line.split('=')
                try:
                    facts[key.strip()] = int(val.strip())
                except:
                    facts[key.strip()] = val.strip()
            elif 'is' in line:
                key, val = line.split('is')
                facts[key.strip()] = val.strip()
            else:
                facts[line.strip()] = True
    return facts, target

def parse_condition(cond):
    cond = cond.strip()
    if '>=' in cond:
        key, val = cond.split('>=')
        return (key.strip(), '>=', float(val.strip()))
    elif '<=' in cond:
        key, val = cond.split('<=')
        return (key.strip(), '<=', float(val.strip()))
    if '>' in cond:
        key, val = cond.split('>')
        return (key.strip(), '>', float(val.strip()))
    elif '<' in cond:
        key, val = cond.split('<')
        return (key.strip(), '<', float(val.strip()))
    elif '=' in cond:
        key, val = cond.split('=')
        try:
            return (key.strip(), '=', int(val.strip()))
        except:
            return (key.strip(), '=', val.strip())
    elif 'is' in cond:
        key, val = cond.split('is')
        return (key.strip(), 'is', val.strip())
    else:
        return (cond.strip(), '==', True)

def parse_rules(file_path):
    rules = []
    with open(file_path, 'r') as f:
        for line in f:
            line = line.strip()
            if not line or 'IF' not in line or 'THEN' not in line:
                continue
            cond_part, conclusion = line.split('THEN')
            cond_part = cond_part.replace('IF', '').strip()
            conclusion = conclusion.strip()

            and_conditions = [c.strip() for c in re.split(r'\bAND\b', cond_part)]
            condition_list = []
            for cond in and_conditions:
                if 'OR' in cond:
                    or_parts = [c.strip() for c in cond.split('OR')]
                    or_group = [parse_condition(c) for c in or_parts]
                    condition_list.append(('OR', or_group))
                else:
                    condition_list.append(parse_condition(cond))
            conclusion_parsed = parse_condition(conclusion)
            rules.append((condition_list, conclusion_parsed))
    return rules

def evaluate_condition(facts, cond):
    key, op, val = cond
    if key not in facts:
        return False
    try:
        if op == '=' or op == 'is':
            return facts[key] == val
        elif op == '==':
            return bool(facts[key])
        elif op == '>':
            return float(facts[key]) > float(val)
        elif op == '<':
            return float(facts[key]) < float(val)
        elif op == '>=':
            return float(facts[key]) >= float(val)
        elif op == '<=':
            return float(facts[key]) <= float(val)
    except:
        return False
    return False

def evaluate_conditions(facts, conditions):
    for cond in conditions:
        if isinstance(cond, tuple) and cond[0] == 'OR':
            if not any(evaluate_condition(facts, c) for c in cond[1]):
                return False
        else:
            if not evaluate_condition(facts, cond):
                return False
    return True

def forward_chaining(facts, rules, target=None):
    changed = True
    while changed:
        changed = False
        print("Current facts:", facts)
        for conditions, conclusion in rules:
            key, op, val = conclusion
            if key in facts and facts[key] == val:
                continue
            if evaluate_conditions(facts, conditions):
                facts[key] = val
                changed = True
                if target and key == target[0] and facts[key] == target[1]:
                    print(f"\nTarget reached: {key} ")
                    return facts
    return facts

def main():
    facts, target = parse_facts('/Users/mac/Desktop/FCAI/reasoning/facts.txt')
    rules = parse_rules('/Users/mac/Desktop/FCAI/reasoning/rules.txt')

    print("\n--- Forward Chaining ---")
    final_facts = forward_chaining(facts, rules, target)

    print("\nFinal inferred facts:")
    for fact in final_facts:
        print(f"{fact} = {final_facts[fact]}")

if __name__ == "__main__":
    main()



--- Forward Chaining ---
Current facts: {'seeds': 0, 'diameter': 7, 'skin_smell': True, 'color': 'orange'}
Current facts: {'seeds': 0, 'diameter': 7, 'skin_smell': True, 'color': 'orange', 'perfumed': True, 'size': 'medium'}
Current facts: {'seeds': 0, 'diameter': 7, 'skin_smell': True, 'color': 'orange', 'perfumed': True, 'size': 'medium', 'fruit': 'orange'}

Target reached: citrus_fruit 

Final inferred facts:
seeds = 0
diameter = 7
skin_smell = True
color = orange
perfumed = True
size = medium
fruit = orange
citrus_fruit = True
